# Entrega: Proyecto 
# Integrantes equipo: Sara Carolina Gómez Delgado, Luis Eduardo Robles Jimenez, María Cristina Velázquez García, María Fernanda Zavala López
# Materia: Ciberseguridad

## Parte 1: Herramienta de decodificadores y reglas basado en unos logs

### Ejemplo de logs

In [ ]:
May 08 15:23:45 macbookdemaria techstore_inventory: type=information username=maria event=login
May 08 17:30:05 pcdepedro techstore_inventory: type=information username=pedro event=add_to_cart product=shirt price=25.00
May 10 12:42:23 pclalito techstore_inventory: type=information username=lalito event=login
May 12 12:52:12 pcsarita techstore_inventory: type=information
May 12 12:52:12 Cristinas-macbook techstore_inventory: type=information username=cristina total=50.00

### Código

In [19]:
import xml.etree.ElementTree as ET

rules = '''
<group name="techstore_inventory">
<rule id="131000" level="3">
<decoded_as>techstore_inventory</decoded_as>
<field name="event">cancel_order</field>
<description>The event indicates that an order with ID $(order_id) was cancelled</description>
</rule>

<rule id="131001" level="7">
<decoded_as>techstore_inventory</decoded_as>
<field name="type">error</field>
<description>Seems like something went wrong</description>
</rule>

<rule id="130002" level="3">
<decoded_as>techstore_inventory</decoded_as>
<field name="event">checkout</field>
<description>User $(username) has made a purchase for a total of $(total)</description>
</rule>

<rule id="130003" level="3">
<decoded_as>techstore_inventory</decoded_as>
<field name="event">add_to_cart</field>
<description>User $(username) has added $(product) to the cart</description>
</rule>

<rule id="130004" level="3">
<decoded_as>techstore_inventory</decoded_as>
<field name="event">checkout</field>
<description>User $(username) has made a purchase for a total of $(total)</description>
</rule>
</group>
'''

decoders = '''
<decoder name="techstore_inventory">
<program_name>techstore_inventory</program_name>
</decoder>
 
<decoder name="log_type">
<parent>techstore_inventory</parent>
<regex>type=(\S+) username=(\S+) event=(\S+)</regex>
<order>type, username, event</order>
</decoder>

<decoder name="log_type">
<parent>techstore_inventory</parent>
<regex>type=(\S+) username=(\S+) event=(\S+) status=(\S+)</regex>
<order>type, username, event, status</order>
</decoder>

<decoder name="log_type">
<parent>techstore_inventory</parent>
<regex>type=(\S+) username=(\S+) event=(\S+) order_id=(\S+)</regex>
<order>type, username, event, order_id</order>
</decoder>
 
<decoder name="log_type">
<parent>techstore_inventory</parent>
<regex>type=(\S+) username=(\S+) event=(\S+) total=(\S+)</regex>
<order>type, username, event, total</order>
</decoder>
 
<decoder name="log_type">
<parent>techstore_inventory</parent>
<regex>type=(\S+) username=(\S+) event=(\S+) product=(\S+) price=(\S+)</regex>
<order>type, username, event, product, price</order>
</decoder>
'''


def setFromXml(xml_string):
    tree = ET.ElementTree(ET.fromstring('<foo>' + xml_string + '</foo>'))
    root = tree.getroot()
    elements = []
    for e in root.iter():
        element = None
        if e.tag == 'regex':
            element = e.text
            elements.append(element)

        if e.tag == 'rule':
            element = {
                'level': e.attrib['level']
            }
            elements.append(element)
        if e.tag == 'field':
            elements[-1]['rule'] = f"{e.attrib['name']}|{e.text}"
        if e.tag == 'description':
            elements[-1]['description'] = e.text
    return elements

with open('logs.log', 'r') as file:
    logs = file.readlines()
logs = [log if log[-1] != '\n' else log[:-1] for log in logs]

e_decoders = setFromXml(decoders)
e_rules = setFromXml(rules)

new_decoders, new_rules = 0, 0
for i, log in enumerate(logs):
    print(f'Log number {i}:', log)
    extra_info = log.split(':')[-1]
    extra_fields = extra_info[1:].split(' ')

    # Build decoder and add it if needed
    decoder = ""
    for field in extra_fields:
        k = field.split('=')[0]
        decoder += f'{k}=(\\S+) '
    decoder = decoder[:-1]
    print(f'\tDecoder is: {decoder}')
    if not decoder in e_decoders:
        e_decoders.append(decoder)
        new_decoders += 1

    # Build rule and add it if needed
    blacklist = ['username', 'product', 'price', 'total']
    for field in extra_fields:
        k, v = field.split('=')
        if k in blacklist: continue
        rule = f'{k}|{v}'
        rules_list = [e['rule'] for e in e_rules]
        if not rule in rules_list:
            description = input(f'\tWrite a description for rule {rule} or n to skip: ')
            if description == 'n': continue
            e_rules.append({
                'level': '5',
                'rule': rule,
                'description': description,
            })
            new_rules += 1

print(f'Added {new_decoders} new decoders')
if new_decoders:
    decoders_file = '<decoder name="techstore_inventory">\n\t<program_name>techstore_inventory</program_name>\n</decoder>'
    for d in e_decoders:
        #order = "".join(d.split("=(\\S+)"))
        order_elements = d.split("=(\\S+)")[:-1]  # Excluye el último elemento vacío
        order = ', '.join(order_elements)  # Une los elementos con comas
        decoders_file += f'\n<decoder name = "log_type">'
        decoders_file += f'\n\t<parent>techstore_inventory</parent>'
        decoders_file += f'\n\t<regex>{d}</regex>'
        decoders_file += f'\n\t<order>{order}</order>'
        decoders_file += f'\n</decoder>'
    print(decoders_file)

print(f'Added {new_rules} new rules')
if new_rules:
    rules_file = '<group name="techstore_inventory">'
    id = 131000
    for r in e_rules:
        k, v = r['rule'].split('|')
        rules_file += f'\n\t<rule id = {id} level = {r["level"]}>'
        rules_file += f'\n\t\t<decoded_as>techstore_inventory</decoded_as>'
        rules_file += f'\n\t\t<field name="{k}">{v}</field>'
        rules_file += f'\n\t\t<description>{r["description"]}</description>'
        rules_file += f'\n\t</rule>'
        id += 1
    rules_file += '\n</group>'
    print(rules_file)


<>:37: SyntaxWarning: invalid escape sequence '\S'
<>:37: SyntaxWarning: invalid escape sequence '\S'
/var/folders/bw/21djx2td1v599svn8rn1xl6m0000gn/T/ipykernel_5636/2641823144.py:37: SyntaxWarning: invalid escape sequence '\S'
  decoders = '''


Log number 0: May 08 15:23:45 macbookdemaria techstore_inventory: type=information username=maria event=login
	Decoder is: type=(\S+) username=(\S+) event=(\S+)


	Write a description for rule type|information or n to skip:  n
	Write a description for rule event|login or n to skip:  The user $(username) has logged in


Log number 1: May 08 17:30:05 pcdepedro techstore_inventory: type=information username=pedro event=add_to_cart product=shirt price=25.00
	Decoder is: type=(\S+) username=(\S+) event=(\S+) product=(\S+) price=(\S+)


	Write a description for rule type|information or n to skip:  n


...

Log number 3: May 12 12:52:12 pcsarita techstore_inventory: type=information
	Decoder is: type=(\S+)


	Write a description for rule type|information or n to skip:  n


Log number 4: May 12 12:52:12 Cristinas-macbook techstore_inventory: type=information username=cristina total=50.00
	Decoder is: type=(\S+) username=(\S+) total=(\S+)


	Write a description for rule type|information or n to skip:  n


Added 2 new decoders
<decoder name="techstore_inventory">
	<program_name>techstore_inventory</program_name>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+)</regex>
	<order>type,  username,  event</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+) status=(\S+)</regex>
	<order>type,  username,  event,  status</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+) order_id=(\S+)</regex>
	<order>type,  username,  event,  order_id</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+) total=(\S+)</regex>
	<order>type,  username,  event,  total</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+) product=(\S+) pri

## Parte 2: Integración con la API de Wazuh

En esta sección lo que se aplica es la conexión con la API.
En cada uno de los pasos es necesaria la autenticación por medio de un usuario y contraseña existentes en el server de Wazuh para poder generar un token. Al recibir el token es posible conectarse con el server de Wazuh y aplicar los cambios u obtener la información correspondiente a cada sección.

### Categoría de la API: Decoders
Finalidad: Se busca que el usuario pueda realizar diversas tareas relacionadas con los decoders. Va a ser capaz de enlistar los archivos de decoders existentes en el server, obtener el contenido de uno de los archivos en particular y finalmente ser capaz de modificar el contenido de un archivo en particular.
Esto es un complemento de la primera parte de nuestro proyecto, ya que es capaz de agregar los decoders que se generaron al usar la herramienta.

#### Tarea de la API: Get files
Return information about all decoders files used in Wazuh. This information include decoder's file, decoder's route and decoder's status among others

In [1]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
decoders_files_endpoint = '/decoders/files'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener los archivos de decoders
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'limit': 500,  # Puedes ajustar este valor según lo necesites
    'sort': '+filename'  # Ordena los resultados por nombre de archivo en orden ascendente
}
decoders_files_response = requests.get(f'{url}{decoders_files_endpoint}', headers=headers, params=params, verify=False)
if decoders_files_response.status_code == 200:
    decoders_files = decoders_files_response.json()
    print("Información de archivos de decoders:")
    for decoder_file in decoders_files['data']['affected_items']:
        print(f"Archivo: {decoder_file['filename']}, Ruta relativa: {decoder_file['relative_dirname']}, Estado: {decoder_file['status']}")
else:
    print(f"Error al obtener información de archivos de decoders: {decoders_files_response.status_code}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Información de archivos de decoders:
Archivo: 0005-wazuh_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0006-json_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0007-wazuh-api_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0010-active-response_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0015-aix-ipsec_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0025-apache_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0030-arpwatch_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0035-asterisk_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0040-auditd_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0045-barracuda_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0050-checkpoint_decoders.xml, Ruta relativa: ruleset/decoders, Estado: enabled
Archivo: 0051-ch

#### Tarea de la API: Get decoders file content
Get the content of a specified decoder file

In [2]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
decoders_files_endpoint = '/decoders/files'
username = 'test_API'
password = '39F{W70EYyQd'
filename = 'fzavalal.xml'  # Asegúrate de sustituir esto por el nombre real del archivo

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener el contenido de un archivo de decoders específico
headers = {'Authorization': f'Bearer {jwt_token}'}
decoder_file_endpoint = f'{decoders_files_endpoint}/{filename}'  # Endpoint para obtener un archivo específico
decoder_file_response = requests.get(f'{url}{decoder_file_endpoint}', headers=headers, verify=False)
if decoder_file_response.status_code == 200:
    decoder_file = decoder_file_response.json()
    print("Contenido del archivo de decoders:")
    for item in decoder_file['data']['affected_items']:
        for decoder in item['decoder']:
            print(f"Nombre del decoder: {decoder['@name']}")
            if 'prematch' in decoder:
                print(f"Prematch: {decoder['prematch']}")
            if 'regex' in decoder:
                print(f"Regex: {decoder['regex']}")
            if 'order' in decoder:
                print(f"Orden: {decoder['order']}")
else:
    print(f"Error al obtener el contenido del archivo de decoders: {decoder_file_response.status_code}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Contenido del archivo de decoders:
Nombre del decoder: techstore_inventory
Nombre del decoder: log_type
Regex: type=(\S+) username=(\S+) event=(\S+)
Orden: type, username, event
Nombre del decoder: log_type
Regex: type=(\S+) username=(\S+) event=(\S+) status=(\S+)
Orden: type, username, event, status
Nombre del decoder: log_type
Regex: type=(\S+) username=(\S+) event=(\S+) order_id=(\S+)
Orden: type, username, event, order_id
Nombre del decoder: log_type
Regex: type=(\S+) username=(\S+) event=(\S+) total=(\S+)
Orden: type, username, event, total
Nombre del decoder: log_type
Regex: type=(\S+) username=(\S+) event=(\S+) product=(\S+) price=(\S+)
Orden: type, username, event, product, price


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Update decoders file
Upload or replace a user decoder file content

In [51]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
decoders_files_endpoint = '/decoders/files'
username = 'test_API'
password = '39F{W70EYyQd'
filename = 'fzavalal.xml'  # Nombre del archivo de decoder que deseas actualizar

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Nuevo contenido del decoder en formato XML como string
new_decoder_content = """
<decoder name="techstore_inventory">
	<program_name>techstore_inventory</program_name>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+)</regex>
	<order>type,  username,  event</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+) status=(\S+)</regex>
	<order>type,  username,  event,  status</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+) order_id=(\S+)</regex>
	<order>type,  username,  event,  order_id</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+) total=(\S+)</regex>
	<order>type,  username,  event,  total</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) event=(\S+) product=(\S+) price=(\S+)</regex>
	<order>type,  username,  event,  product,  price</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+)</regex>
	<order>type</order>
</decoder>
<decoder name = "log_type">
	<parent>techstore_inventory</parent>
	<regex>type=(\S+) username=(\S+) total=(\S+)</regex>
	<order>type,  username,  total</order>
</decoder>
"""

# Endpoint para cargar/reemplazar un archivo específico
decoder_file_endpoint = f'{decoders_files_endpoint}/{filename}?overwrite=true'

# Usar el token JWT para cargar/reemplazar el contenido del archivo de decoder
headers = {
    'Authorization': f'Bearer {jwt_token}',
    'Content-Type': 'application/octet-stream'  # Cambiado a application/octet-stream
}

# Convertir el contenido XML a bytes para enviar como application/octet-stream
byte_content = new_decoder_content.encode('utf-8')

# Realizar la petición PUT
put_response = requests.put(f'{url}{decoder_file_endpoint}', headers=headers, data=byte_content, verify=False)
if put_response.status_code == 200:
    print("El contenido del decoder fue actualizado exitosamente.")
    print(put_response.json())  
else:
    print(f"Error al actualizar el archivo de decoder: {put_response.status_code}")
    print(put_response.text)  


<>:21: SyntaxWarning: invalid escape sequence '\S'
<>:21: SyntaxWarning: invalid escape sequence '\S'
/var/folders/bw/21djx2td1v599svn8rn1xl6m0000gn/T/ipykernel_5636/4174930421.py:21: SyntaxWarning: invalid escape sequence '\S'
  new_decoder_content = """
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
El contenido del decoder fue actualizado exitosamente.
{'data': {'affected_items': ['etc/decoders/fzavalal.xml'], 'total_affected_items': 1, 'total_failed_items': 0, 'failed_items': []}, 'message': 'Decoder was successfully uploaded', 'error': 0}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Categoría de la API: Rules
Finalidad: Se busca que el usuario pueda realizar diversas tareas relacionadas con las rules. Va a ser capaz de enlistar los archivos de rules existentes en el server, obtener el contenido de uno de los archivos en particular y finalmente ser capaz de modificar el contenido de un archivo en particular.
Esto es un complemento de la primera parte de nuestro proyecto, ya que es capaz de agregar las reglas que se generaron al usar la herramienta.

#### Tarea de la API: Get files
Return a list containing all files used to define rules and their status

In [52]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
rules_files_endpoint = '/rules/files'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener los archivos de rules
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'limit': 500,  
    'sort': '+filename'  # Ordena los resultados por nombre de archivo en orden ascendente
}
rule_files_response = requests.get(f'{url}{rules_files_endpoint}', headers=headers, params=params, verify=False)
if rule_files_response.status_code == 200:
    rules_files = rule_files_response.json()
    print("Información de archivos de rules:")
    for rule_file in rules_files['data']['affected_items']:
        print(f"Archivo: {rule_file['filename']}, Ruta relativa: {rule_file['relative_dirname']}, Estado: {rule_file['status']}")
else:
    print(f"Error al obtener información de archivos de decoders: {rules_files_response.status_code}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Información de archivos de rules:
Archivo: 0010-rules_config.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0015-ossec_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0016-wazuh_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0017-wazuh-api_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0020-syslog_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0025-sendmail_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0030-postfix_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0035-spamd_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0040-imapd_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0045-mailscanner_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0050-ms-exchange_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0055-courier_rules.xml, Ruta relativa: ruleset/rules, Estado: enabled
Archivo: 0065-pix

#### Tarea de la API: Get rules file content
Get the content of a specified rule in the ruleset

In [54]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
rules_files_endpoint = '/rules/files'
username = 'test_API'
password = '39F{W70EYyQd'
filename = 'fzavalal.xml'  # Nombre del archivo de reglas que deseas ver

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener el contenido de un archivo de reglas específico
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'true',
    'raw': 'true'  # Obtener el contenido en texto plano
}
rule_file_endpoint = f'{rules_files_endpoint}/{filename}'  # Endpoint con el nombre del archivo específico
rule_content_response = requests.get(f'{url}{rule_file_endpoint}', headers=headers, params=params, verify=False)
if rule_content_response.status_code == 200:
    print("Contenido del archivo de reglas:")
    print(rule_content_response.text)  
else:
    print(f"Error al obtener el contenido del archivo de reglas: {rule_content_response.status_code}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Contenido del archivo de reglas:

<group name="techstore_inventory">
	<rule id="131000" level="3">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="event">cancel_order</field>
		<description>The event indicates that an order with ID $(order_id) was cancelled</description>
	</rule>
	<rule id="131001" level="7">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="type">error</field>
		<description>Seems like something went wrong</description>
	</rule>
	<rule id="131002" level="3">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="event">checkout</field>
		<description>User $(username) has made a purchase for a total of $(total)</description>
	</rule>
	<rule id="131003" level="3">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="event">add_to_cart</field>
		<description>User $(username) has added $(product) to the cart</description>
	</rule>
	<rule id="131004" level="3">
		<decoded_as>techstore_inventory</de

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Update rules file
Upload or replace a user ruleset file content

In [56]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
rules_files_endpoint = '/rules/files'
username = 'test_API'
password = '39F{W70EYyQd'
filename = 'fzavalal.xml'  # Nombre del archivo de reglas que deseas actualizar

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Nuevo contenido del archivo de reglas en formato XML como string
new_rules_content = """
<group name="techstore_inventory">
	<rule id="131000" level="3">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="event">cancel_order</field>
		<description>The event indicates that an order with ID $(order_id) was cancelled</description>
	</rule>
	<rule id="131001" level="7">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="type">error</field>
		<description>Seems like something went wrong</description>
	</rule>
	<rule id="131002" level="3">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="event">checkout</field>
		<description>User $(username) has made a purchase for a total of $(total)</description>
	</rule>
	<rule id="131003" level="3">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="event">add_to_cart</field>
		<description>User $(username) has added $(product) to the cart</description>
	</rule>
	<rule id="131004" level="3">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="event">checkout</field>
		<description>User $(username) has made a purchase for a total of $(total)</description>
	</rule>
	<rule id="131005" level="5">
		<decoded_as>techstore_inventory</decoded_as>
		<field name="event">login</field>
		<description>The user $(username) has logged in</description>
	</rule>
</group>

"""

# Convertir el contenido XML a bytes para enviar como application/octet-stream
byte_content = new_rules_content.encode('utf-8')

# Endpoint para cargar/reemplazar un archivo específico
rule_file_endpoint = f'{rules_files_endpoint}/{filename}?overwrite=true'

# Usar el token JWT para cargar/reemplazar el contenido del archivo de reglas
headers = {
    'Authorization': f'Bearer {jwt_token}',
    'Content-Type': 'application/octet-stream'  
}

# Realizar la petición PUT
put_response = requests.put(f'{url}{rule_file_endpoint}', headers=headers, data=byte_content, verify=False)
if put_response.status_code == 200:
    print("El contenido del archivo de reglas fue actualizado exitosamente.")
    print(put_response.json())  
else:
    print(f"Error al actualizar el archivo de reglas: {put_response.status_code}")
    print(put_response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
El contenido del archivo de reglas fue actualizado exitosamente.
{'data': {'affected_items': ['etc/rules/fzavalal.xml'], 'total_affected_items': 1, 'total_failed_items': 0, 'failed_items': []}, 'message': 'Rule was successfully uploaded', 'error': 0}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Recuerda:
Cada que se aplica una regla nueva es necesario reiniciar el manager. No te preocupes, este notebook puede ayudarte a realizar eso también. Solamente debes ir a la sección de Manager y buscar la tarea que permite reiniciar el manager, una vez que corras esa celda, el manager estará listo.

### Categoría de la API: Manager
Finalidad: Se busca que el usuario pueda realizar diversas tareas relacionadas con el manager. Se va a poder el estatus de todos los daemons de Wazuh, obtener los logs más recientes y también reiniciar el manager. 
Esto es un complemento de la primera parte de nuestro proyecto, ya que cada que se añade una regla más al server es necesario reiniciar el manager para que se vea aplicada.

#### Tarea de la API: Get status
Return the status of all Wazuh daemons

In [58]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
manager_status_endpoint = '/manager/status'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener el estado del manager
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'true',  
    'wait_for_complete': 'false'  
}
manager_status_response = requests.get(f'{url}{manager_status_endpoint}', headers=headers, params=params, verify=False)
if manager_status_response.status_code == 200:
    manager_status = manager_status_response.json()
    print("Estado de los demonios de Wazuh:")
    for daemon, status in manager_status['data']['affected_items'][0].items():
        print(f"{daemon}: {status}")
else:
    print(f"Error al obtener el estado del manager: {manager_status_response.status_code}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Estado de los demonios de Wazuh:
wazuh-agentlessd: stopped
wazuh-analysisd: running
wazuh-authd: running
wazuh-csyslogd: stopped
wazuh-dbd: stopped
wazuh-monitord: running
wazuh-execd: running
wazuh-integratord: running
wazuh-logcollector: running
wazuh-maild: stopped
wazuh-remoted: running
wazuh-reportd: stopped
wazuh-syscheckd: running
wazuh-clusterd: stopped
wazuh-modulesd: running
wazuh-db: running
wazuh-apid: running


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Get logs
Return the last wazuh log entries

In [59]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
logs_endpoint = '/manager/logs'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Configuración de headers y parámetros iniciales
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'false',
    'wait_for_complete': 'true',  
    'limit': 500,  
    'sort': '-timestamp',  # Ordena de forma descendente por timestamp
}

# Obtener hasta 2000 logs realizando múltiples solicitudes si es necesario
all_logs = []
max_logs = 2000
total_received = 0
while total_received < max_logs:
    logs_response = requests.get(f'{url}{logs_endpoint}', headers=headers, params=params, verify=False)
    if logs_response.status_code == 200:
        logs_data = logs_response.json()
        all_logs.extend(logs_data['data']['affected_items'])
        received = len(logs_data['data']['affected_items'])
        total_received += received
        if received < 500:  # Si se reciben menos de 500, significa que no hay más logs que obtener
            break
    else:
        print(f"Error al obtener los logs: {logs_response.status_code}")
        if logs_response.text:
            print("Detalle del error:", logs_response.text)  
        break


print("Logs obtenidos con éxito:")
for item in all_logs:
    print(f"Timestamp: {item['timestamp']}, Tag: {item['tag']}, Level: {item['level']}, Description: {item['description']}")




/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/adv

Logs obtenidos con éxito:
Timestamp: 2024-05-13T02:51:03Z, Tag: wazuh-analysisd, Level: error, Description:  (1226): Error reading XML file 'etc/decoders/fzavalal.xml': XMLERR: Attribute 'name' has no value. (line 5).
Timestamp: 2024-05-13T02:51:03Z, Tag: wazuh-analysisd, Level: critical, Description:  (1202): Configuration error at 'etc/decoders/fzavalal.xml'.
Timestamp: 2024-05-13T02:50:58Z, Tag: wazuh-analysisd, Level: error, Description:  (1226): Error reading XML file 'etc/decoders/fzavalal.xml': XMLERR: Attribute 'name' has no value. (line 5).
Timestamp: 2024-05-13T02:50:58Z, Tag: wazuh-analysisd, Level: critical, Description:  (1202): Configuration error at 'etc/decoders/fzavalal.xml'.
Timestamp: 2024-05-13T02:50:18Z, Tag: wazuh-analysisd, Level: error, Description:  (1226): Error reading XML file 'etc/decoders/fzavalal.xml': XMLERR: Attribute 'name' has no value. (line 5).
Timestamp: 2024-05-13T02:50:18Z, Tag: wazuh-analysisd, Level: critical, Description:  (1202): Configuratio

#### Tarea de la API: Get logs summary
Return a summary of the last wazuh log entries

In [1]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
summary_logs_endpoint = '/manager/logs/summary'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener el resumen de los logs
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'false',
    'wait_for_complete': 'true' 
}

# Realizar la solicitud de resumen de logs
summary_response = requests.get(f'{url}{summary_logs_endpoint}', headers=headers, params=params, verify=False)
if summary_response.status_code == 200:
    summary_data = summary_response.json()
    print("Resumen de logs obtenido con éxito:")
    for item in summary_data['data']['affected_items']:
        for module, stats in item.items():
            print(f"Module: {module}")
            print(f" Info: {stats.get('info', 0)}")
            print(f" All: {stats.get('all', 0)}")
            print(f" Critical: {stats.get('critical', 0)}")
            print(f" Debug: {stats.get('debug', 0)}")
            print(f" Error: {stats.get('error', 0)}")
            print(f" Warning: {stats.get('warning', 0)}")
else:
    print(f"Error al obtener el resumen de logs: {summary_response.status_code}")
    if summary_response.text:
        print("Detalle del error:", summary_response.text)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Resumen de logs obtenido con éxito:
Module: sca
 Info: 90
 All: 90
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-agentlessd
 Info: 15
 All: 15
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-analysisd
 Info: 75
 All: 162
 Critical: 42
 Debug: 0
 Error: 45
Module: wazuh-authd
 Info: 83
 All: 87
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-csyslogd
 Info: 15
 All: 15
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-db
 Info: 32
 All: 32
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-dbd
 Info: 15
 All: 15
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-execd
 Info: 45
 All: 45
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-integratord
 Info: 45
 All: 45
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-logcollector
 Info: 150
 All: 150
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-modulesd
 Info: 15
 All: 21
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-modulesd:agent-upgrade
 Info: 15
 All: 15
 Critical: 0
 Debug: 0
 Error: 0
Module: wazuh-modulesd:ciscat

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Restart manager
Restart the wazuh manager

In [1]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
restart_endpoint = '/manager/restart'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para enviar la solicitud de reinicio al manager
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'false',
    'wait_for_complete': 'true'  
}
restart_response = requests.put(f'{url}{restart_endpoint}', headers=headers, params=params, verify=False)
if restart_response.status_code == 200:
    print("El reinicio del manager ha sido exitosamente solicitado.")
    print(restart_response.json())  
else:
    print(f"Error al solicitar el reinicio del manager: {restart_response.status_code}")
    if restart_response.text:
        print("Detalle del error:", restart_response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
El reinicio del manager ha sido exitosamente solicitado.
{'data': {'affected_items': ['manager'], 'total_affected_items': 1, 'total_failed_items': 0, 'failed_items': []}, 'message': 'Restart request sent to  all specified nodes', 'error': 0}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Categoría de la API: Security
Finalidad: Se busca que el usuario pueda realizar diversas tareas relacionadas con la seguridad. Principalmente se proporcionan tareas las cuales pueden ayudar en el manejo de los usuarios en Wazuh. Es con las credenciales de estos usuarios que se puede obtener el token necesario para poder realizar todas las tareas de este notebook, por lo cual el que sea posible obtener información del usuario que se usa actualmente, obtener un listado de los usuarios, añadir un usuario, eliminarlo, poder cambiar su password, enlistar los roles de cada usuario y finalmente agregar o quitar roles por usuario nos permite tener mayor facilidad para realizar diversos cambios en Wazuh.
Esto es un complemento de la primera parte de nuestro proyecto, ya que al poder manejar los usuarios es que nos es posible realizar tareas como modificar los archivos de los decoders y las reglas.

#### Tarea de la API: Get current user info
Get the information of the current user

In [2]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
current_user_info_endpoint = '/security/users/me'
username = 'test_API'
password = '39F{W70EYyQd'
#username = 'testcvelazquez'
#password = 'PassTesting2025{'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener la información del usuario actual
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'false',
    'wait_for_complete': 'false'
}
user_info_response = requests.get(f'{url}{current_user_info_endpoint}', headers=headers, params=params, verify=False)
if user_info_response.status_code == 200:
    user_info_data = user_info_response.json()
    user_info = user_info_data['data']['affected_items'][0] if user_info_data['data']['affected_items'] else {}
    print("Información del usuario actual:")
    print(f"ID: {user_info.get('id')}, Username: {user_info.get('username')}")
    print("Roles y políticas:")
    for role in user_info.get('roles', []):
        print(f"Role ID: {role['id']}, Name: {role['name']}")
        for policy in role.get('policies', []):
            print(f"Policy ID: {policy['id']}, Name: {policy['name']}, Policy Details: {policy['policy']}")
else:
    print(f"Error al obtener la información del usuario actual: {user_info_response.status_code}")
    if user_info_response.text:
        print("Detalle del error:", user_info_response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Información del usuario actual:
ID: 102, Username: test_API
Roles y políticas:
Role ID: 1, Name: administrator
Policy ID: 1, Name: agents_all_resourceless, Policy Details: {'actions': ['agent:create', 'group:create'], 'resources': ['*:*:*'], 'effect': 'allow'}
Policy ID: 2, Name: agents_all_agents, Policy Details: {'actions': ['agent:read', 'agent:delete', 'agent:modify_group', 'agent:reconnect', 'agent:restart', 'agent:upgrade'], 'resources': ['agent:id:*', 'agent:group:*'], 'effect': 'allow'}
Policy ID: 3, Name: agents_all_groups, Policy Details: {'actions': ['group:read', 'group:delete', 'group:update_config', 'group:modify_assignments'], 'resources': ['group:id:*'], 'effect': 'allow'}
Policy ID: 6, Name: agents_commands_agents, Policy Details: {'actions': ['active-response:command'], 'resources': ['agent:id:*'], 'effect': 'allow'}
Policy ID: 7, Name: security_all_resourceless, Policy Details: {'actions': ['security:create', 'security:create_user', 'security:read_config', 'security:

#### Tarea de la API: List users
Get the information of a specified user

In [3]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
users_endpoint = '/security/users'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener la lista de usuarios
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'false',  
    'limit': 500,  
    'sort': 'id'  # Ordenar por ID de usuario
}
users_response = requests.get(f'{url}{users_endpoint}', headers=headers, params=params, verify=False)
if users_response.status_code == 200:
    users_data = users_response.json()
    print("Lista de usuarios:")
    for user in users_data['data']['affected_items']:
        print(f"ID: {user.get('id')}, Username: {user['username']}, Allow run as: {user['allow_run_as']}, Roles: {user['roles']}")
else:
    print(f"Error al obtener la lista de usuarios: {users_response.status_code}")
    if users_response.text:
        print("Detalle del error:", users_response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Lista de usuarios:
ID: 1, Username: wazuh, Allow run as: True, Roles: [1]
ID: 2, Username: wazuh-wui, Allow run as: True, Roles: [1]
ID: 100, Username: admin, Allow run as: True, Roles: [1]
ID: 101, Username: test_user, Allow run as: False, Roles: [1]
ID: 102, Username: test_API, Allow run as: True, Roles: [1]
ID: 103, Username: wazuh_pd, Allow run as: True, Roles: [1]
ID: 104, Username: wazuh_perro_dinamita, Allow run as: True, Roles: [1]
ID: 105, Username: lalito, Allow run as: True, Roles: [1, 3]
ID: 106, Username: test_afjrr, Allow run as: False, Roles: [1]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Add user
Add a new API user to the system

In [4]:
import requests
import json

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
add_user_endpoint = '/security/users'
admin_username = 'test_API'  # Usuario existente con privilegios suficientes para agregar usuarios
admin_password = '39F{W70EYyQd'

# Detalles del nuevo usuario
new_username = "testcvelazquez"  # Cambia esto al nombre de usuario deseado
new_password = "PassTesting2024{"  # Cambia esto a una contraseña segura

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(admin_username, admin_password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Datos del nuevo usuario
user_data = {
    "username": new_username,
    "password": new_password
}

# Headers para la solicitud
headers = {
    'Authorization': f'Bearer {jwt_token}',
    'Content-Type': 'application/json'
}

# Realizar la solicitud POST para agregar el nuevo usuario
response = requests.post(f'{url}{add_user_endpoint}', headers=headers, data=json.dumps(user_data), verify=False)
if response.status_code == 200:
    print("Nuevo usuario agregado exitosamente.")
    print(response.json())  
else:
    print(f"Error al agregar el usuario: {response.status_code}")
    print(response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Nuevo usuario agregado exitosamente.
{'data': {'affected_items': [{'id': 107, 'username': 'testcvelazquez', 'allow_run_as': False, 'roles': []}], 'total_affected_items': 1, 'total_failed_items': 0, 'failed_items': []}, 'message': 'User was successfully created', 'error': 0}


#### Tarea de la API: Update users
Modify a user's password by specifying their ID

In [38]:
import requests
import json

def authenticate(username, password):
    """ Autenticación y obtención del token JWT. """
    url = 'https://54.149.137.67:55000/security/user/authenticate'
    auth_response = requests.post(url, auth=(username, password), verify=False)
    if auth_response.status_code == 200:
        return auth_response.json()['data']['token']
    else:
        print(f"Error al autenticarse: {auth_response.status_code}")
        exit()

def get_users(token):
    """ Obtener la lista de todos los usuarios. """
    url = 'https://54.149.137.67:55000/security/users'
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers, verify=False)
    return response.json()['data']['affected_items'] if response.status_code == 200 else []

def find_user_id(users, username):
    """ Buscar y devolver el ID del usuario por su nombre. """
    for user in users:
        if user['username'] == username:
            return user['id']
    return None

def update_user_password(token, user_id, new_password):
    """ Modificar el password de un usuario. """
    url = f'https://54.149.137.67:55000/security/users/{user_id}'
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    data = json.dumps({"password": new_password})
    response = requests.put(url, headers=headers, data=data, verify=False)
    return response

# Datos para la autenticación
admin_username = 'test_API'
admin_password = '39F{W70EYyQd'

# Autenticación y obtención del token JWT
jwt_token = authenticate(admin_username, admin_password)

# Obtener la lista de usuarios
users = get_users(jwt_token)

# Entrada del nombre del usuario
target_username = input("Introduce el nombre del usuario cuyo password deseas modificar: ")

# Buscar el ID del usuario por nombre
user_id = find_user_id(users, target_username)

if user_id:
    print(f"Usuario encontrado: ID {user_id}")
    # Entrada del nuevo password
    new_password = input("Introduce el nuevo password para el usuario: ")

    # Modificar el password del usuario
    response = update_user_password(jwt_token, user_id, new_password)
    if response.status_code == 200:
        print("Password modificado correctamente.")
        print(response.json())
    else:
        print(f"Error al modificar el password del usuario: {response.status_code}")
        print(response.text)
else:
    print("Usuario no encontrado.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Introduce el nombre del usuario cuyo password deseas modificar:  testcvelazquez


Usuario encontrado: ID 107


Introduce el nuevo password para el usuario:  PassTesting2025{


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Password modificado correctamente.
{'data': {'affected_items': [{'id': 107, 'username': 'testcvelazquez', 'allow_run_as': False, 'roles': []}], 'total_affected_items': 1, 'total_failed_items': 0, 'failed_items': []}, 'message': 'User was successfully updated', 'error': 0}


#### Tarea de la API: List roles
For a specific list, indicate the ids separated by commas. Example: ?role_ids=1,2,3

In [5]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
roles_endpoint = '/security/roles'
username = 'test_API'  
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener la lista de roles
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'false',
    'limit': 500,  
    'sort': 'id'  # Ordenar por ID de role
}
roles_response = requests.get(f'{url}{roles_endpoint}', headers=headers, params=params, verify=False)
if roles_response.status_code == 200:
    roles_data = roles_response.json()
    print("Lista de roles:")
    for role in roles_data['data']['affected_items']:
        print(f"ID: {role['id']}, Name: {role['name']}")
        print(f"  Policies: {role['policies']}")
        print(f"  Users: {role['users']}")
        print(f"  Rules: {role['rules']}")
else:
    print(f"Error al obtener la lista de roles: {roles_response.status_code}")
    if roles_response.text:
        print("Detalle del error:", roles_response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Lista de roles:
ID: 1, Name: administrator
  Policies: [1, 2, 3, 6, 7, 8, 29, 30, 12, 14, 15, 18, 19, 21, 23, 24, 16, 25, 27, 28, 33, 34, 35, 36, 37]
  Users: [1, 2, 100, 101, 102, 103, 104, 105, 106]
  Rules: [1, 2]
ID: 2, Name: readonly
  Policies: [4, 5, 12, 31, 32, 13, 17, 20, 22, 16, 25, 26, 28, 35]
  Users: []
  Rules: []
ID: 3, Name: users_admin
  Policies: [9, 10]
  Users: [105]
  Rules: []
ID: 4, Name: agents_readonly
  Policies: [4, 5]
  Users: []
  Rules: []
ID: 5, Name: agents_admin
  Policies: [1, 2, 3]
  Users: []
  Rules: []
ID: 6, Name: cluster_readonly
  Policies: [31, 32]
  Users: []
  Rules: []
ID: 7, Name: cluster_admin
  Policies: [29, 30]
  Users: []
  Rules: []


#### Tarea personificada (no se encuentra en la API): Enlistar los roles que presentan los usuarios

In [6]:
import requests

def authenticate(url, username, password):
    """Autentica y obtiene el token JWT."""
    auth_url = f'{url}/security/user/authenticate'
    response = requests.post(auth_url, auth=(username, password), verify=False)
    if response.status_code == 200:
        return response.json()['data']['token']
    else:
        raise Exception(f"Error al autenticarse: {response.status_code}")

def get_roles(url, token, role_ids):
    """Obtiene detalles de roles por IDs."""
    roles_endpoint = f'{url}/security/roles'
    headers = {'Authorization': f'Bearer {token}'}
    params = {'role_ids': ','.join(map(str, role_ids))}
    response = requests.get(roles_endpoint, headers=headers, params=params, verify=False)
    if response.status_code == 200:
        return {role['id']: role['name'] for role in response.json()['data']['affected_items']}
    else:
        print("Error al obtener detalles de roles")
        return {}

def get_users_with_roles(url, token):
    """Obtiene usuarios y sus roles asociados."""
    users_endpoint = f'{url}/security/users'
    headers = {'Authorization': f'Bearer {token}'}
    users_response = requests.get(users_endpoint, headers=headers, verify=False)

    if users_response.status_code == 200:
        users_data = users_response.json()['data']['affected_items']
        for user in users_data:
            print(f"ID: {user['id']}, Username: {user['username']}, Allow run as: {user['allow_run_as']}")

            if user['roles']:
                # Obtener los nombres de los roles usando sus IDs
                role_names = get_roles(url, token, user['roles'])
                print("  Roles:")
                for role_id in user['roles']:
                    role_name = role_names.get(role_id, 'Role name not found')
                    print(f"    Role ID: {role_id}, Role Name: {role_name}")
            else:
                print("  Roles: No tiene roles asignados")
    else:
        print(f"Error al obtener la lista de usuarios: {users_response.status_code}")
        if users_response.text:
            print("Detalle del error:", users_response.text)

# Configuración de la API
url = 'https://54.149.137.67:55000'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticación y obtención del token JWT
jwt_token = authenticate(url, username, password)

# Obtener y mostrar usuarios con sus roles
get_users_with_roles(url, jwt_token)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ID: 1, Username: wazuh, Allow run as: True


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


  Roles:
    Role ID: 1, Role Name: administrator
ID: 2, Username: wazuh-wui, Allow run as: True


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


  Roles:
    Role ID: 1, Role Name: administrator
ID: 105, Username: lalito, Allow run as: True


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


  Roles:
    Role ID: 1, Role Name: administrator
    Role ID: 3, Role Name: users_admin
ID: 106, Username: test_afjrr, Allow run as: False
  Roles:
    Role ID: 1, Role Name: administrator
ID: 107, Username: testcvelazquez, Allow run as: False
  Roles: No tiene roles asignados


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Add roles to user
Create a specified relation role-policy, one user may have multiples roles

In [7]:
import requests
import json

def authenticate(username, password):
    """ Autenticación y obtención del token JWT. """
    url = 'https://54.149.137.67:55000/security/user/authenticate'
    auth_response = requests.post(url, auth=(username, password), verify=False)
    if auth_response.status_code == 200:
        return auth_response.json()['data']['token']
    else:
        print(f"Error al autenticarse: {auth_response.status_code}")
        exit()

def get_users(token):
    """ Obtener la lista de todos los usuarios. """
    url = 'https://54.149.137.67:55000/security/users'
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers, verify=False)
    return response.json()['data']['affected_items'] if response.status_code == 200 else []

def find_user_id(users, username):
    """ Buscar y devolver el ID del usuario por su nombre. """
    for user in users:
        if user['username'] == username:
            return user['id']
    return None

def get_roles(token):
    """ Obtener la lista de roles disponibles. """
    url = 'https://54.149.137.67:55000/security/roles'
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers, verify=False)
    return response.json()['data']['affected_items'] if response.status_code == 200 else []

def add_roles_to_user(token, user_id, role_ids):
    """ Añadir roles seleccionados a un usuario específico usando parámetros de consulta. """
    url = f'https://54.149.137.67:55000/security/users/{user_id}/roles'
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    params = {'role_ids': ','.join(role_ids)}  # Envío de role_ids como parámetro de consulta
    response = requests.post(url, headers=headers, params=params, verify=False)
    return response

# Datos para la autenticación
admin_username = 'test_API'
admin_password = '39F{W70EYyQd'

# Autenticación y obtención del token JWT
jwt_token = authenticate(admin_username, admin_password)

# Obtener la lista de usuarios
users = get_users(jwt_token)

# Entrada del nombre del usuario
target_username = input("Introduce el nombre del usuario al que deseas añadir roles: ")

# Buscar el ID del usuario por nombre
user_id = find_user_id(users, target_username)

if user_id:
    print(f"Usuario encontrado: ID {user_id}")
    # Obtener y mostrar roles disponibles
    roles = get_roles(jwt_token)
    print("ID\tNombre del Rol")
    for role in roles:
        print(f"{role['id']}\t{role['name']}")
    
    selected_roles = input("Introduce los IDs de los roles a añadir, separados por comas: ").split(',')
    selected_roles = [role.strip() for role in selected_roles if role.strip().isdigit()]

    # Añadir roles al usuario
    response = add_roles_to_user(jwt_token, user_id, selected_roles)
    if response.status_code == 200:
        print("Roles añadidos correctamente al usuario.")
        print(response.json())
    else:
        print(f"Error al añadir roles al usuario: {response.status_code}")
        print(response.text)
else:
    print("Usuario no encontrado.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Introduce el nombre del usuario al que deseas añadir roles:  testcvelazquez


Usuario encontrado: ID 107


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ID	Nombre del Rol
1	administrator
2	readonly
3	users_admin
4	agents_readonly
5	agents_admin
6	cluster_readonly
7	cluster_admin


Introduce los IDs de los roles a añadir, separados por comas:  1,2,3


Roles añadidos correctamente al usuario.
{'data': {'affected_items': [{'id': 107, 'username': 'testcvelazquez', 'allow_run_as': False, 'roles': [1, 2, 3]}], 'total_affected_items': 3, 'total_failed_items': 0, 'failed_items': []}, 'message': 'All roles were linked to user testcvelazquez', 'error': 0}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea API: Remove roles from user
Delete a specified relation user-roles

In [8]:
import requests
import json

def authenticate(url, username, password):
    """Autentica y obtiene el token JWT."""
    auth_url = f'{url}/security/user/authenticate'
    response = requests.post(auth_url, auth=(username, password), verify=False)
    if response.status_code == 200:
        return response.json()['data']['token']
    else:
        raise Exception(f"Error al autenticarse: {response.status_code}")

def get_roles(url, token, role_ids):
    """Obtiene detalles de roles por IDs."""
    roles_endpoint = f'{url}/security/roles'
    headers = {'Authorization': f'Bearer {token}'}
    params = {'role_ids': ','.join(map(str, role_ids))}
    response = requests.get(roles_endpoint, headers=headers, params=params, verify=False)
    if response.status_code == 200:
        return {role['id']: role['name'] for role in response.json()['data']['affected_items']}
    else:
        print("Error al obtener detalles de roles")
        return {}

def get_user_by_name(url, token, username):
    """Obtiene un usuario por nombre y muestra sus roles."""
    users_endpoint = f'{url}/security/users'
    headers = {'Authorization': f'Bearer {token}'}
    users_response = requests.get(users_endpoint, headers=headers, verify=False)

    if users_response.status_code == 200:
        users_data = users_response.json()['data']['affected_items']
        user = next((u for u in users_data if u['username'] == username), None)
        if user:
            print(f"ID: {user['id']}, Username: {user['username']}, Allow run as: {user['allow_run_as']}")
            if user['roles']:
                role_names = get_roles(url, token, user['roles'])
                print("  Roles:")
                for role_id in user['roles']:
                    role_name = role_names.get(role_id, 'Role name not found')
                    print(f"    Role ID: {role_id}, Role Name: {role_name}")
                return user['id'], user['roles']
            else:
                print("  Roles: No tiene roles asignados")
                return user['id'], []
        else:
            print("Usuario no encontrado.")
            return None, None
    else:
        print(f"Error al obtener la lista de usuarios: {users_response.status_code}")
        if users_response.text:
            print("Detalle del error:", users_response.text)
        return None, None

def remove_roles_from_user(url, token, user_id, role_ids):
    """Elimina roles de un usuario."""
    roles_endpoint = f'{url}/security/users/{user_id}/roles'
    headers = {'Authorization': f'Bearer {token}'}
    params = {'role_ids': ','.join(map(str, role_ids))}
    response = requests.delete(roles_endpoint, headers=headers, params=params, verify=False)
    if response.status_code == 200:
        print("Roles eliminados correctamente.")
        print(response.json())
    else:
        print(f"Error al eliminar roles: {response.status_code}")
        if response.text:
            print("Detalle del error:", response.text)

# Configuración de la API
url = 'https://54.149.137.67:55000'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticación y obtención del token JWT
jwt_token = authenticate(url, username, password)

# Entrada de nombre del usuario
target_username = input("Introduce el nombre del usuario para ver y eliminar roles: ")

# Obtener usuario y roles
user_id, current_role_ids = get_user_by_name(url, jwt_token, target_username)

# Proceso de eliminación de roles si el usuario existe
if user_id:
    role_ids_to_remove = input("Introduce los IDs de los roles a eliminar, separados por comas: ").split(',')
    remove_roles_from_user(url, jwt_token, user_id, role_ids_to_remove)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Introduce el nombre del usuario para ver y eliminar roles:  testcvelazquez


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ID: 107, Username: testcvelazquez, Allow run as: False


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


  Roles:
    Role ID: 1, Role Name: administrator
    Role ID: 2, Role Name: readonly
    Role ID: 3, Role Name: users_admin


Introduce los IDs de los roles a eliminar, separados por comas:  1,2


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Roles eliminados correctamente.
{'data': {'affected_items': [{'id': 107, 'username': 'testcvelazquez', 'allow_run_as': False, 'roles': [3]}], 'total_affected_items': 2, 'total_failed_items': 0, 'failed_items': []}, 'message': 'All roles were unlinked from user testcvelazquez', 'error': 0}


#### Tarea de la API: Delete users
Delete a list of users by specifying their IDs

In [9]:
import requests

def authenticate(username, password):
    """ Autenticación y obtención del token JWT. """
    url = 'https://54.149.137.67:55000/security/user/authenticate'
    auth_response = requests.post(url, auth=(username, password), verify=False)
    if auth_response.status_code == 200:
        return auth_response.json()['data']['token']
    else:
        print(f"Error al autenticarse: {auth_response.status_code}")
        exit()

def get_users(token):
    """ Obtener la lista de todos los usuarios. """
    url = 'https://54.149.137.67:55000/security/users'
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers, verify=False)
    return response.json()['data']['affected_items'] if response.status_code == 200 else []

def find_user_id(users, username):
    """ Buscar y devolver el ID del usuario por su nombre. """
    for user in users:
        if user['username'] == username:
            return user['id']
    return None

def delete_user(token, user_id):
    """ Eliminar un usuario específico por su ID. """
    url = f'https://54.149.137.67:55000/security/users'
    headers = {'Authorization': f'Bearer {token}'}
    params = {'user_ids': user_id}
    response = requests.delete(url, headers=headers, params=params, verify=False)
    return response

# Datos para la autenticación
admin_username = 'test_API'
admin_password = '39F{W70EYyQd'

# Autenticación y obtención del token JWT
jwt_token = authenticate(admin_username, admin_password)

# Obtener la lista de usuarios
users = get_users(jwt_token)

# Entrada del nombre del usuario a eliminar
target_username = input("Introduce el nombre del usuario que deseas eliminar: ")

# Buscar el ID del usuario por nombre
user_id = find_user_id(users, target_username)

if user_id:
    print(f"Usuario encontrado: ID {user_id}")
    # Eliminar el usuario
    response = delete_user(jwt_token, user_id)
    if response.status_code == 200:
        print("Usuario eliminado correctamente.")
        print(response.json())
    else:
        print(f"Error al eliminar el usuario: {response.status_code}")
        print(response.text)
else:
    print("Usuario no encontrado.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Introduce el nombre del usuario que deseas eliminar:  testcvelazquez


Usuario encontrado: ID 107
Usuario eliminado correctamente.
{'data': {'affected_items': [{'id': 107, 'username': 'testcvelazquez', 'allow_run_as': False, 'roles': [3]}], 'total_affected_items': 1, 'total_failed_items': 0, 'failed_items': []}, 'message': 'Users were successfully deleted', 'error': 0}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Nota:
Con las previas tareas se facilita todo lo relacionado al manejo de decoders y reglas. Con la primera sección del notebook se pueden crear y a partir de la segunda sección se pueden manipular los archivos relacionados con los decoders y las reglas. Las tareas extras, como las de manager y security nos permiten realizar varias tareas, pero lo principal de ellas es que nos permiten reiniciar el manager (cosa que se necesita para que se reflejen las nuevas reglas en Wazuh) y controlar los usuarios existentes en Wazuh (es importante por que sin usuarios y sus credenciales no sería posible correr los códigos que permiten el manejo de archivos de decoders y reglas).

A partir de aquí en adelante se presentan tareas relacionadas a los agentes. No está directamente relacionado con los decodificadores y reglas pero nos permiten obtener información que puede ser relevante sobre Wazuh.

### Categoría de la API: Agents
Finalidad: Se busca que el usuario pueda realizar diversas tareas relacionadas con los agentes. Ejemplos de esto es: enlistarlos, eliminarlos, añadir nuevos agentes, obtener la llave de los agentes y un resumen de los agentes.

#### Tarea de la API: List agents
Return information about all available agents or a list of them

In [10]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
agents_endpoint = '/agents'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener la lista de agentes
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'false',
    'wait_for_complete': 'false',
    'limit': 500,  
    'sort': 'id',  # Ordenar por ID del agente
    'status': 'active'  # Filtrar agentes por estado, ejemplos: 'active', 'disconnected', etc.
}
agents_response = requests.get(f'{url}{agents_endpoint}', headers=headers, params=params, verify=False)
if agents_response.status_code == 200:
    agents_data = agents_response.json()
    print("Lista de agentes:")
    for agent in agents_data['data']['affected_items']:
        print(f"ID: {agent['id']}, Name: {agent['name']}, IP: {agent['ip']}, Status: {agent['status']}")
else:
    print(f"Error al obtener la lista de agentes: {agents_response.status_code}")
    if agents_response.text:
        print("Detalle del error:", agents_response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Lista de agentes:
ID: 000, Name: ip-172-31-43-44, IP: 127.0.0.1, Status: active
ID: 006, Name: MSI, IP: 192.168.1.72, Status: active
ID: 011, Name: DESKTOP-JLLA54H, IP: 192.168.5.192, Status: active
ID: 013, Name: DESKTOP-VV0NSUQ, IP: 192.168.0.121, Status: active
ID: 020, Name: DESKTOP-M36ACQR, IP: 192.168.0.20, Status: active


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Listado de agentes sin importar estatus

In [11]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
agents_endpoint = '/agents'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener la lista de agentes
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'false',
    'wait_for_complete': 'false',
    'limit': 500,  
    'sort': 'id',  # Ordenar por ID del agente
    'status': 'active,disconnected,never_connected,pending'  # Incluir todos los estados posibles
}
agents_response = requests.get(f'{url}{agents_endpoint}', headers=headers, params=params, verify=False)
if agents_response.status_code == 200:
    agents_data = agents_response.json()
    print("Lista completa de agentes:")
    for agent in agents_data['data']['affected_items']:
        print(f"ID: {agent['id']}, Name: {agent['name']}, IP: {agent['ip']}, Status: {agent['status']}")
else:
    print(f"Error al obtener la lista completa de agentes: {agents_response.status_code}")
    if agents_response.text:
        print("Detalle del error:", agents_response.text) 


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Lista completa de agentes:
ID: 000, Name: ip-172-31-43-44, IP: 127.0.0.1, Status: active
ID: 001, Name: MacBook-Pro-de-Maria.local, IP: 192.168.1.71, Status: disconnected
ID: 002, Name: Cristinas-MacBook-Pro.local, IP: 192.168.1.80, Status: disconnected
ID: 003, Name: 5e4865fgulihjopkl, IP: 192.168.3.31, Status: disconnected
ID: 005, Name: AlanSalazar, IP: 192.168.1.66, Status: disconnected
ID: 006, Name: MSI, IP: 192.168.1.72, Status: active
ID: 007, Name: Josue0H-Laptop, IP: 192.168.1.88, Status: disconnected
ID: 008, Name: DESKTOP-5SQ6K0T, IP: 192.168.100.29, Status: disconnected
ID: 009, Name: LAPTOP-H91C5K4C, IP: 192.168.68.106, Status: disconnected
ID: 010, Name: FabStation, IP: 172.20.10.7, Status: disconnected
ID: 011, Name: DESKTOP-JLLA54H, IP: 192.168.5.192, Status: active
ID: 012, Name: DESKTOP-0Q36ERP, IP: 192.168.113.253, Status: disconnected
ID: 013, Name: DESKTOP-VV0NSUQ, IP: 192.168.0.121, Status: active
ID: 014, Name: JHludwolf.local, IP: 192.168.1.98, Status: disconne

#### Tarea de la API: Add agent
Add a new agent

In [12]:
import requests
import json

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
agents_endpoint = '/agents'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Datos del nuevo agente
new_agent_data = {
    "name": "NewHost_2",  # Nombre del nuevo agente
    #"ip": "10.0.10.11"    # Opcional: dirección IP del agente
}

# Headers para la solicitud
headers = {
    'Authorization': f'Bearer {jwt_token}',
    'Content-Type': 'application/json'
}

# Realizar la solicitud POST para agregar el nuevo agente
response = requests.post(f'{url}{agents_endpoint}', headers=headers, data=json.dumps(new_agent_data), verify=False)
if response.status_code == 200:
    print("Agente agregado exitosamente.")
    print(response.json())  
else:
    print(f"Error al agregar el agente: {response.status_code}")
    print(response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Agente agregado exitosamente.
{'data': {'id': '025', 'key': 'MDI1IE5ld0hvc3RfMiBhbnkgYjAyZjk0OTZlMjNhYjNjYWE4MjgxYTlmNjVkMTY3NTQ5MzEzMTM0MTljMjI3ODg2YjlhNmM4ZDNmYmI5MTBmZg=='}, 'error': 0}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Delete agents
Delete all agents or a list of them based on optional criteria

In [13]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
agents_endpoint = '/agents'
username = 'test_API'
password = '39F{W70EYyQd'
agent_name_to_delete = "NewHost_2"  # Asegúrate de ajustar al nombre real del agente

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Encontrar el ID del agente por su nombre
headers = {'Authorization': f'Bearer {jwt_token}'}
search_params = {
    'name': agent_name_to_delete,
    'status': 'active,disconnected,never_connected'
}
search_response = requests.get(f'{url}{agents_endpoint}', headers=headers, params=search_params, verify=False)
if search_response.status_code == 200:
    search_data = search_response.json()
    agents_found = search_data['data']['affected_items']
    if agents_found:
        agent_ids = ','.join([agent['id'] for agent in agents_found])
        print(f"IDs encontrados para el nombre '{agent_name_to_delete}': {agent_ids}")

        # Eliminar los agentes encontrados
        delete_params = {
            'agents_list': agent_ids,
            'purge': 'true',
            'status': 'active,disconnected,never_connected',
            'older_than': '0s'  # Ajuste clave para incluir todos los agentes
        }
        delete_response = requests.delete(f'{url}{agents_endpoint}', headers=headers, params=delete_params, verify=False)
        if delete_response.status_code == 200:
            print("Agentes eliminados exitosamente.")
            print(delete_response.json())
        else:
            print(f"Error al eliminar agentes: {delete_response.status_code}")
            print(delete_response.text)
    else:
        print("No se encontraron agentes con ese nombre.")
else:
    print(f"Error al buscar agentes: {search_response.status_code}")
    print(search_response.text)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


IDs encontrados para el nombre 'NewHost_2': 025
Agentes eliminados exitosamente.
{'data': {'affected_items': ['025'], 'total_affected_items': 1, 'total_failed_items': 0, 'failed_items': []}, 'message': 'All selected agents were deleted', 'error': 0}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Get key
Return the key of an agent

In [3]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
agents_endpoint = '/agents'
key_endpoint = '/agents/{agent_id}/key'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener la lista de agentes
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'false',
    'wait_for_complete': 'false',
    'limit': 500,  
    'sort': 'id',  # Ordenar por ID del agente
    'status': 'active,disconnected,never_connected,pending'  # Incluir todos los estados posibles
}
agents_response = requests.get(f'{url}{agents_endpoint}', headers=headers, params=params, verify=False)
if agents_response.status_code == 200:
    agents_data = agents_response.json()
    print("Lista completa de agentes y sus claves:")
    for agent in agents_data['data']['affected_items']:
        # Para cada agente, obtener su clave
        agent_key_response = requests.get(f'{url}{key_endpoint.format(agent_id=agent["id"])}', headers=headers, verify=False)
        if agent_key_response.status_code == 200:
            agent_key_data = agent_key_response.json()
            key = agent_key_data['data']['affected_items'][0]['key'] if agent_key_data['data']['affected_items'] else "No key available"
            print(f"ID: {agent['id']}, Name: {agent['name']}, IP: {agent['ip']}, Status: {agent['status']}, Key: {key}")
        else:
            print(f"Error al obtener la llave del agente con ID: {agent['id']}: {agent_key_response.status_code}")
else:
    print(f"Error al obtener la lista completa de agentes: {agents_response.status_code}")
    if agents_response.text:
        print("Detalle del error:", agents_response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Lista completa de agentes y sus claves:


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ID: 022, Name: NewAgent, IP: any, Status: never_connected, Key: MDIyIE5ld0FnZW50IGFueSA0ODUyY2I2ODA1MTIwNDk2ODMyMTEwYzVjOTg4M2I1OTFhYzU1Y2Q4M2U2MmI2NDlmMmMxODI3MmI4Y2ZhMTk4


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ID: 023, Name: unknown-device, IP: 192.168.0.117, Status: disconnected, Key: MDIzIHVua25vd24tZGV2aWNlIGFueSBhYzg3NmFiNjAwOGNlNDdhYmEzNWQyZjA4NjMwMGQ0ZDZjMTYxMDcwNjMzMDFkNjRlYjhlN2FlMjA4ZmM3YjU1
ID: 024, Name: lap-25870, IP: 192.168.5.9, Status: disconnected, Key: MDI0IGxhcC0yNTg3MCBhbnkgMDY2ZDgxODI5MDlkNjc1MzhkZjgyZjc1NGZmMTYwN2MzZmZmMzY4ZmIwNmJjZjYxZDcyNmM4MjYzZmNjOWJiMg==


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Summarize agents status
Return a summary of the connection and groups configuration synchronization statuses of available agents

In [4]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
summary_status_endpoint = '/agents/summary/status'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener el resumen del estado de los agentes
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'true',  
    'wait_for_complete': 'true'  
}
summary_status_response = requests.get(f'{url}{summary_status_endpoint}', headers=headers, params=params, verify=False)
if summary_status_response.status_code == 200:
    summary_status_data = summary_status_response.json()
    print("Resumen de estado de los agentes:")
    print(f"Conexión - Activos: {summary_status_data['data']['connection']['active']}, "
          f"Desconectados: {summary_status_data['data']['connection']['disconnected']}, "
          f"Nunca Conectados: {summary_status_data['data']['connection']['never_connected']}, "
          f"Pendientes: {summary_status_data['data']['connection']['pending']}, "
          f"Total: {summary_status_data['data']['connection']['total']}")
    print(f"Configuración - Sincronizados: {summary_status_data['data']['configuration']['synced']}, "
          f"No Sincronizados: {summary_status_data['data']['configuration']['not_synced']}, "
          f"Total: {summary_status_data['data']['configuration']['total']}")
else:
    print(f"Error al obtener el resumen del estado de los agentes: {summary_status_response.status_code}")
    if summary_status_response.text:
        print("Detalle del error:", summary_status_response.text)  


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Resumen de estado de los agentes:
Conexión - Activos: 4, Desconectados: 15, Nunca Conectados: 1, Pendientes: 0, Total: 20
Configuración - Sincronizados: 19, No Sincronizados: 1, Total: 20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Categoría de la API: Overview
Finalidad: Se busca que el usuario pueda obtener mayor información de los agentes.

#### Tarea de la API: Get agents overview
Return a dictionary with a full agents overview

In [6]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
agents_overview_endpoint = '/overview/agents'
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Usar el token JWT para obtener el resumen de los agentes
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'true',
    'wait_for_complete': 'true'  
}

# Realizar la solicitud de resumen de los agentes
overview_response = requests.get(f'{url}{agents_overview_endpoint}', headers=headers, params=params, verify=False)
if overview_response.status_code == 200:
    overview_data = overview_response.json()['data']
    print("Resumen de agentes obtenido con éxito:")
    print(f"Nodes: {overview_data['nodes']}")
    print(f"Groups: {overview_data['groups']}")
    print(f"Agent OS: {overview_data['agent_os']}")
    print(f"Agent Status: {overview_data['agent_status']}")
    print(f"Agent Version: {overview_data['agent_version']}")
    print(f"Last Registered Agent: {overview_data['last_registered_agent']}")
else:
    print(f"Error al obtener el resumen de agentes: {overview_response.status_code}")
    if overview_response.text:
        print("Detalle del error:", overview_response.text)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Resumen de agentes obtenido con éxito:
Nodes: [{'count': 19, 'node_name': 'node01'}, {'count': 1, 'node_name': 'unknown'}]
Groups: [{'name': 'default', 'count': 19, 'mergedSum': '4a8724b20dee0124ff9656783c490c4e', 'configSum': 'ab73af41699f13fdd81903b5f23d8d00'}]
Agent OS: [{'os': {'name': 'macOS', 'platform': 'darwin', 'version': '14.4.1'}, 'count': 3}, {'os': {'name': 'Microsoft Windows 11 Home Single Language', 'platform': 'windows', 'version': '10.0.22631.3447'}, 'count': 3}, {'os': {'name': 'Microsoft Windows 10 Home Single Language', 'platform': 'windows', 'version': '10.0.19045.4291'}, 'count': 2}, {'os': {'name': 'Microsoft Windows 11 Home', 'platform': 'windows', 'version': '10.0.22631.3527'}, 'count': 3}, {'os': {'name': 'Microsoft Windows 10 Pro', 'platform': 'windows', 'version': '10.0.19045.4046'}, 'count': 1}, {'os': {'name': 'Microsoft Windows 11 Home', 'platform': 'windows', 'version': '10.0.22631.3447'}, 'count': 1}, {'os': {'name': 'M

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Categoría de la API: Syscollector
Finalidad: Se busca que el usuario pueda obtener mayor información de los agentes, como lo es el hardware, netaddr y su sistema operativo

#### Tarea de la API: Get agent hardware
Return the agent's hardware info. This information include cpu, ram, scan info among others

In [9]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
hardware_endpoint = '/syscollector/{agent_id}/hardware'  
agent_id = '023'  # ID del agente
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Configurar el endpoint con el ID del agente
hardware_endpoint = hardware_endpoint.replace("{agent_id}", agent_id)

# Usar el token JWT para obtener la información de hardware del agente
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'true',
    'wait_for_complete': 'true'  
}

# Realizar la solicitud para obtener información del hardware del agente
hardware_response = requests.get(f'{url}{hardware_endpoint}', headers=headers, params=params, verify=False)
if hardware_response.status_code == 200:
    hardware_data = hardware_response.json()
    print("Información de hardware obtenida con éxito:")
    for item in hardware_data['data']['affected_items']:
        print(f"CPU: {item['cpu']}")
        print(f"RAM: {item['ram']}")
        print(f"Scan Info: {item['scan']}")
        print(f"Board Serial: {item['board_serial']}")
        print(f"Agent ID: {item['agent_id']}")
else:
    print(f"Error al obtener la información de hardware: {hardware_response.status_code}")
    if hardware_response.text:
        print("Detalle del error:", hardware_response.text)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Información de hardware obtenida con éxito:
CPU: {'cores': 8, 'mhz': 1800, 'name': 'Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz'}
RAM: {'free': 1798348, 'total': 8218788, 'usage': 78}
Scan Info: {'id': 0, 'time': '2024-04-02T02:36:54+00:00'}
Board Serial: PHHZCBA0GB60BZ
Agent ID: 023


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Get agent netaddr
Return the agent's network address info. This information include used IP protocol, interface, IP address among others

In [13]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
network_endpoint = '/syscollector/{agent_id}/netaddr'  
agent_id = '023'  # ID del agente
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Configurar el endpoint con el ID del agente
network_endpoint = network_endpoint.replace("{agent_id}", agent_id)

# Usar el token JWT para obtener la información de red del agente
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'true',
    'wait_for_complete': 'true'  
}

# Realizar la solicitud para obtener información de red del agente
network_response = requests.get(f'{url}{network_endpoint}', headers=headers, params=params, verify=False)
if network_response.status_code == 200:
    network_data = network_response.json()
    print("Información de la dirección de red obtenida con éxito:")
    for item in network_data['data']['affected_items']:
        print(f"Interface: {item['iface']}")
        print(f"IP Address: {item['address']}")
        print(f"Broadcast: {item['broadcast']}")
        print(f"Protocol: {item['proto']}")
else:
    print(f"Error al obtener la información de red: {network_response.status_code}")
    if network_response.text:
        print("Detalle del error:", network_response.text)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Información de la dirección de red obtenida con éxito:
Interface: Conexión de red Bluetooth
IP Address: fe80::8953:cfdc:cfb5:e6b8
Broadcast:  
Protocol: ipv6
Interface: PdaNet Broadband Connection
IP Address: fe80::9646:c3e9:5dc2:fda9
Broadcast:  
Protocol: ipv6
Interface: Loopback Pseudo-Interface 1
IP Address: 127.0.0.1
Broadcast: 127.255.255.255
Protocol: ipv4
Interface: Conexión de red Bluetooth
IP Address: 169.254.155.8
Broadcast: 169.254.255.255
Protocol: ipv4
Interface: Loopback Pseudo-Interface 1
IP Address: ::1
Broadcast:  
Protocol: ipv6
Interface: Conexión de área local* 1
IP Address: fe80::c9e6:c558:3872:be1b
Broadcast:  
Protocol: ipv6
Interface: PdaNet Broadband Connection
IP Address: 169.254.91.73
Broadcast: 169.254.255.255
Protocol: ipv4
Interface: Conexión de área local* 1
IP Address: 169.254.234.31
Broadcast: 169.254.255.255
Protocol: ipv4
Interface: Wi-Fi
IP Address: fe80::5f42:37f3:3361:9ac4
Broadcast:  
Protocol: ipv6
Interface: Co

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


#### Tarea de la API: Get agent OS
Return the agent's OS info. This information include os information, architecture information among others of all agents

In [15]:
import requests

# Configuración de la API
url = 'https://54.149.137.67:55000'
auth_endpoint = '/security/user/authenticate'
os_info_endpoint = '/syscollector/{agent_id}/os' 
agent_id = '023'  # ID del agente
username = 'test_API'
password = '39F{W70EYyQd'

# Autenticarse para obtener el token JWT
auth_response = requests.post(f'{url}{auth_endpoint}', auth=(username, password), verify=False)
if auth_response.status_code == 200:
    jwt_token = auth_response.json()['data']['token']
    print("Token JWT obtenido correctamente.")
else:
    print(f"Error al autenticarse: {auth_response.status_code}")
    exit()

# Configurar el endpoint con el ID del agente
os_info_endpoint = os_info_endpoint.replace("{agent_id}", agent_id)

# Usar el token JWT para obtener la información del sistema operativo del agente
headers = {'Authorization': f'Bearer {jwt_token}'}
params = {
    'pretty': 'true',
    'wait_for_complete': 'true' 
}

# Realizar la solicitud para obtener información del sistema operativo del agente
os_response = requests.get(f'{url}{os_info_endpoint}', headers=headers, params=params, verify=False)
if os_response.status_code == 200:
    os_data = os_response.json()
    print("Información del sistema operativo obtenida con éxito:")
    for item in os_data['data']['affected_items']:
        print(f"OS Name: {item['os']['name']}, Version: {item['os']['version']}")
        print(f"Architecture: {item['architecture']}")
        print(f"Hostname: {item['hostname']}")
else:
    print(f"Error al obtener la información del sistema operativo: {os_response.status_code}")
    if os_response.text:
        print("Detalle del error:", os_response.text)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Token JWT obtenido correctamente.
Información del sistema operativo obtenida con éxito:
OS Name: Microsoft Windows 11 Home Single Language, Version: 10.0.22631.3374
Architecture: x86_64
Hostname: UNKNOWN-DEVICE


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '54.149.137.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
